In [1]:
import pandas as pd
from urllib.parse import urlparse
import socket
import idna
import re
from bs4 import BeautifulSoup
from collections import Counter
import requests

In [2]:
df=pd.read_csv('phishing_site_urls.csv')
df

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad
...,...,...
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad


### dodajemy length_url

In [3]:
def length_URL(url):
    l=len(url)
    return l

In [4]:
for i in range(len(df)):
    df.loc[i, "length_url"] = length_URL(df.loc[i, "URL"])
df

,URL,Label,length_url
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0
...,...,...,...
549341,23.227.196.215/,bad,15.0
549342,apple-checker.org/,bad,18.0
549343,apple-iclods.org/,bad,17.0
549344,apple-uptoday.org/,bad,18.0


### dodajemy info o tym czy w URL jest protokół (http lub https)

In [5]:
def is_scheme(url):
    scheme = urlparse(url).scheme
    
    if len(scheme)==0:
        return 0
    else:
        return 1

In [6]:
for i in range(len(df)):
    df.loc[i, "scheme"] = is_scheme(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0
...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0
549342,apple-checker.org/,bad,18.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0


### dodajemy length_hostname

In [7]:
def length_hostname(url):
    # Parse the URL to extract the hostname
    hostname = urlparse(url).hostname
    if hostname == None:
        modified='http://'+url
        hostname = urlparse(modified).hostname
        if hostname== None:
            l=0
        else:
            l=len(hostname)
    else:
        l=len(hostname)
    return l

Naprawa kilku URL (usunięcie znaków "[" oraz "]")

In [8]:
df.loc[18245,'URL']='\x8aRybjUx\x18Ùãl5»7ÆE%Ý\x14Ôk+h\x1f\x0f|U\x1a\x07+ýk©ìÉ\x94½\x93ÆqâF·\x9eõÁ¢w)ëA·ç\x93\x0f°\x9a{t*m!¦2\x03'
df.loc[18277,'URL']='\x90Æe\x1eF§÷%\x11¶\x1c¿Õ\x8c½9¿b@Ö¸ÚZE¤ÒC¢\x98\x8eÄÅª2åç-W³fU¤\x1eJgkz.ø¿n\x80Jçå\x14æu\x87\x85øD%@ðû\x0fÇùM¹u\x0bË'
df.loc[18278,'URL']='\x90Æe\x1eF§÷%\x11¶\x1c¿Õ\x8c½9¿b@Ö¸ÚZE¤ÒC¢\x98\x8eÄÅª2åç-W³fU¤\x1eJgkz.ø¿n\x80Jçå\x14æu\x87\x85øD%@ðû\x0fÇùM¹u\x0bË'
df.loc[18283,'URL']='\x90Æe\x1eF§÷%\x11¶\x1c¿Õ\x8c½9¿b@Ö¸ÚZE¤ÒC¢\x98\x8eÄÅª2åç-W³fU¤\x1eJgkz.ø¿n\x80Jçå\x14æu\x87\x85øD%@ðû\x0fÇùM¹u\x0bË'
df.loc[18288,'URL']='¨R\x98ÊÃ\x86ûaCóÞit×ßÂe-DÖ\x8bØ+9YèÌçÏ\x97¯·\x04"0£ÙÕ.0ößF«7¹N\x89R\x1c\x04Ù{ccÉÄãéçxÄ6a\x1a5Ñ³LÖíÜÉÀ£\x9dÒma¥yRX\x03\x9a*0ÅÝ7×Ê\x83ÁÌ\x05\x05o«Õs¶\x8d0k\x90dèÑ&\x83\x1cÄ\x10"Ï¨mZ\'àD\x8fM×ñ\x01XÚÒK"päî±h¬\x83cAÊeK@4r"^\'ÓFþ1*Ë\x8e\x1d\x8dË PÞô;õ\x9c$úàÑ@\x87þ=êWÑ"Ãhñ\x05\x18®ç^\x18\x11«Ýó^ç\x1a\x1fRú\x8eUJ\x14.<6C\x19\x94y\x1a\x9fÜ\x94FØrÿV2ôæý\x89\x03Zãii\x16\x93I\x0e\x8ab;\x13\x16¨Ë\x97\x9cµu^Í\x99V\x90y)\x9d\xadè»âýº\x01+\x9f\x94S\x99Ö\x1e\x17á\x10\x95\x03Ãì?\x1få6åÔ/'
df.loc[18296,'URL']='1Î¼0#W»æ½Î4>¥õ\x1cª\x94(\\xl\x863(ò5?¹(\x8d°åþ¬eéÍû\x12\x06µÆÒÒ-&\x92\x1d\x14Äv&-Q\x97/9jê½\x9b2\xad òS;ÑwÅût\x02W?(§3¬</Â!*\x07\x87Ø~?ÊmË¨^XV\x9c¹µÂ\x92¦\x183¨|÷4\x10fÈë<\t\x81ô·»n³H\x96\x99éÜúÂÒá/Wîà.K3q4:å\x81)¿®I\x13K.°x±\x8e&\x0fR6\x87\x90¹àÄ\x01#\x1f|9³¢Ü\x89\x87\x94ù\x94ñ\x14\x19~3'
df.loc[18303,'URL']='k\x8d¥¤ZM$:)\x88ìLZ£.^rÕÕ{6eZAä¦v·¢ï\x8böè\x04®~QNgXx_\x84BT\x0f\x0f°ü\x7f\x94Û\x97P¼°\xad9sk\x85%L0gPø·îh Í\x91Öx\x13\x03éovÝf\x91-3Ó¹õ\x85¥Â^¯ÝÀ\\\x96fâhuË\x02S\x7f\\\x92&\x96`ñc\x1cL\x1e¤m\x0f!sÁ\x88\x02F>øsgE¹\x13\x0f)ó)â(2üf\x15ã).!÷\x0f\x92ÿÞ<rDZÅ*¼\x8b/\xa0e¼Ëh\x97\x01úW\x1d+\x06\x9b\x05ï%»»;µÛ\x13\x9e¦M5;ù\\¸¥ß\x9fãV«û°\x8bz¦ö9Ì\'\x06Î\x03|\xa0¼ôªz"8#1¿\x04D4A /.|qt}Òåo#Ûõ\x14*|Bv\x0c®\x18U¬\x9e¢'
df.loc[18304,'URL']="A\x0eìfÙêìÝÕ\x1d2\x87£»\x91¸ü\x19\x909nªÉ®'\x97A½`ym\x9aî¹èDéI5\x08û-ÄXå\x10¦\x01²\x90Â%js²ÍD^\x85^\x89\x9e\x15Á*\x8f\x89^Ü\x98\x86D-À\x18\x91%³;Óßñg\\8+±Wnn@\x83¾IßBëC\x8c³3S7\x9bM(úJzª6¥\x93\x18HmÒ)fæ(\x01þ\x17i\x1e\x13¡d\xa0\x16ù\x8dp"
df.loc[18312,'URL']='½\x13<+\x82U\x1f\x12½¹1\x0c\x88\x801"Kfw§¿âÎ¸pWb®ÜÜ\x81\x07|\x93¾\x85Ö\x87\x18·ff¦o6\x9aQõF\x94õTmK&0\x90Û¤RÍÌP\x03ü.Ò<\'BÉ@-ó\x1aàYN\x01\\\x9a¦~7J\x85¡\x82*\x9aÈú=ÙU\xa0^>R~@O·\x8c'
df.loc[18314,'URL']='¯\x04\x88=\x84ÓÛ±\x13i-\x11Ð\x18'
df.loc[18315,'URL']='=\x9dRã\x0fmôj³{\x94è\x95!ÀM\x97¶6<\x9cN>\x9ew\x85¼Cf\x11£\x1b4ÍnÝÌ\x9c'
df.loc[18316,'URL']='9Ý&Ö\x1aW\x95%\x809¢£\x0cÓ}|¨<\x1cÀPVú½W;hÓ\x06¹\x06N\x8d\x12h1ÌA\x02'
df.loc[18319,'URL']='ò\x82óÒ©è\x88à\x8cÆü\x11\x10Ñ\x04\x80½l¹ñÅÑ÷K\x95¼\x8foÔP©ñ\tØ2¸aV²z\x885\x04;±\x9bg«³wTtÊ\x1e\x8eîFãðf@åº«&¸\x9e\x06õ\x81å¶kºç¡\x13¥$Ô#ì·\x11c\x94B\x98\x06ÊC\x08\x95©ÎË5\x11z\x15z&xW\x04ª>%{rb\x19\x10·'

In [9]:
for i in range(len(df)):
    df.loc[i, "length_hostname"] = length_hostname(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0
...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0
549342,apple-checker.org/,bad,18.0,0.0,17.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0


### dodajemy ip

def ip(url):
    hostname = urlparse(url).hostname
    if hostname == None:
        modified='http://'+url
        hostname = urlparse(modified).hostname
        ip_address = socket.gethostbyname(hostname)

    else:
        ip_address = socket.gethostbyname(hostname) 

    return ip_address

def ip(url):
    hostname = urlparse(url).hostname
    if hostname is None:
        modified = 'http://' + url
        hostname = urlparse(modified).hostname

    try:
        ip_address = socket.gethostbyname(hostname)
    except socket.gaierror:
        #print(f"Could not resolve hostname: {hostname}")
        return 0
    except idna.IDNAError:
        return 0

    return ip_address

for i in range(len(df)):
    df.loc[i, "ip"] = ip(df.loc[i, "URL"])
    
df

i

url='http://'+df["URL"][0]
ip(url)

url='http://'+df["URL"][1]
hostname = urlparse(url).hostname
hostname
ip_address = socket.gethostbyname(hostname)
if ip_address==socket.gaierror:
    ip_address=None

### dodajemy nb_qm - liczbę wystąpień znaku '?' w URL

In [10]:
def nb_qmark(url):
    return url.count('?')

In [11]:
for i in range(len(df)):
    df.loc[i, "nb_qm"] = nb_qmark(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0
...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0


### dodajemy nb_www

In [12]:
def nb_www(url):
    return url.count('www')

In [13]:
for i in range(len(df)):
    df.loc[i, "nb_www"] = nb_www(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0
...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0


### dodajemy ratio_digits_url 

In [14]:
def count_digits_URL(url):
    count = 0
    for char in url:
        if char.isdigit():
            count += 1
    return count

In [15]:
for i in range(len(df)):
    df.loc[i, "count_digits_url"] = count_digits_URL(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0
...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0


In [16]:
df['ratio_digits_url']=df['count_digits_url']/df['length_url']
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034
...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000


### ratio_digits_host

In [17]:
def count_digits_hostname(url):
    # Parse the URL to extract the hostname
    count =0
    hostname = urlparse(url).hostname
    if hostname == None:
        modified='http://'+url
        hostname = urlparse(modified).hostname
        if hostname== None:
            return 0
    
    for char in hostname:
        if char.isdigit():
            count+=1
    return count

In [18]:
for i in range(len(df)):
    df.loc[i, "count_digits_hostname"] = count_digits_hostname(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0
...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0


In [19]:
df['ratio_digits_hostname']=df['count_digits_hostname']/df['length_hostname']
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000


### longest_words_raw

In [20]:
def longest_word_length_in_url(url):
    # Usuń znaki specjalne z URL
    cleaned_url = re.sub(r'[/:?&.=]', ' ', url)
    # Podziel URL na słowa
    words = cleaned_url.split()
    # Znajdź najdłuższe słowo i zwróć jego długość
    if words:
        longest_word = max(words, key=len)
        return len(longest_word)
    else:
        return 0

In [21]:
for i in range(len(df)):
    df.loc[i, "longest_words_raw"] = longest_word_length_in_url(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname,longest_words_raw
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000,35.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000,14.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000,40.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000,15.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714,3.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0


### longest_word_path

In [22]:
def longest_word_length_in_path(url):
    scheme = urlparse(url).scheme
    if scheme=='':
        url='http://'+url
    path = urlparse(url).path
    # Usuń znaki specjalne z path
    cleaned_path = re.sub(r'[/:?&.=]', ' ', path)
    # Podziel path na słowa
    words = cleaned_path.split()
    # Znajdź najdłuższe słowo i zwróć jego długość
    if words:
        longest_word = max(words, key=len)
        return len(longest_word)
    else:
        return 0

In [23]:
for i in range(len(df)):
    df.loc[i, "longest_words_path"] = longest_word_length_in_path(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname,longest_words_raw,longest_words_path
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000,35.0,35.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000,14.0,14.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000,40.0,40.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000,15.0,15.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000,23.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714,3.0,0.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0


### dodajemy phish_hints

### dodajemy nb_hyperlinks

def get_unique_link_count(url):
    try:
        # Pobranie zawartości strony internetowej
        scheme = urlparse(url).scheme
        if scheme=='':
            url='http://'+url
        response = requests.get(url)
        response.raise_for_status()  # sprawdzenie, czy żądanie zakończyło się sukcesem

        # Parsowanie zawartości strony
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Znalezienie wszystkich linków i filtrowanie ich
        foundUrls = Counter([link["href"] for link in soup.find_all("a", href=lambda href: href and not href.startswith("#"))])
        
        # Obliczenie liczby unikalnych URL-i
        count = len(foundUrls)
        return count
    
    except Exception as e:
        # Jeśli wystąpi jakikolwiek błąd, zwróć None
        return None

for i in range(len(df)):
    df.loc[i, "nb_hyperlinks"] = get_unique_link_count(df.loc[i, "URL"])
df

### dodajemy tld_in_subdomain

In [24]:
import tldextract

def check_tld_in_subdomain(url):
    extracted = tldextract.extract(url)
    subdomain = extracted.subdomain
    tld = extracted.suffix

    # Sprawdzenie, czy TLD znajduje się w subdomenie
    if tld in subdomain.split('.'):
        return 1
    return 0

In [39]:
check_tld_in_subdomain(df["URL"][0])

0

In [40]:
for i in range(len(df)):
    df.loc[i, "tld_in_subdomain"] = check_tld_in_subdomain(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname,longest_words_raw,longest_words_path,tld_in_subdomain
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000,35.0,35.0,0.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000,14.0,14.0,0.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000,40.0,40.0,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000,15.0,15.0,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000,23.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714,3.0,0.0,1.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.0,0.0,0.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0


### prefix_suffix

In [41]:
import tldextract

def check_prefix_suffix_in_domain(url):
    extracted = tldextract.extract(url)
    
    subdomain = extracted.subdomain
    domain = extracted.domain
    suffix = extracted.suffix
    
    # Sprawdzenie czy domena ma prefiks (czy subdomena nie jest pusta)
    has_prefix = bool(subdomain)
    
    # Sprawdzenie czy domena ma sufiks (czy suffix nie jest pusty)
    has_suffix = bool(suffix)
    if (has_prefix ==True) or (has_suffix==True):
        return 1
    else:
        return 0

In [42]:
check_prefix_suffix_in_domain(df["URL"][346])

1

In [43]:
for i in range(len(df)):
    df.loc[i, "prefix_suffix"] = check_prefix_suffix_in_domain(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname,longest_words_raw,longest_words_path,tld_in_subdomain,prefix_suffix
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000,35.0,35.0,0.0,1.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000,14.0,14.0,0.0,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000,40.0,40.0,0.0,1.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000,15.0,15.0,0.0,1.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000,23.0,15.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714,3.0,0.0,1.0,0.0
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0,1.0
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.0,0.0,0.0,1.0
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0,1.0


### shortest_word_host

In [57]:
def shortest_host(url):
    hostname = urlparse(url).hostname
    if hostname == None:
        modified='http://'+url
        hostname = urlparse(modified).hostname
        if hostname==None:
            return 0
    # Usuń znaki specjalne z host
    cleaned_hostname = re.sub(r'[/:?&.=]', ' ', hostname)
    # Podziel path na słowa
    words = cleaned_hostname.split()
    # Znajdź najdłuższe słowo i zwróć jego długość
    if words:
        shortest_word = min(words, key=len)
        return len(shortest_word)
    else:
        return 0

In [29]:
shortest_host(df["URL"][346])

3

In [58]:
for i in range(len(df)):
    df.loc[i, "shortest_word_host"] = shortest_host(df.loc[i, "URL"])
df

,URL,Label,length_url,scheme,length_hostname,nb_qm,nb_www,count_digits_url,ratio_digits_url,count_digits_hostname,ratio_digits_hostname,longest_words_raw,longest_words_path,tld_in_subdomain,prefix_suffix,shortest_word_host,empty_title
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,225.0,0.0,9.0,1.0,0.0,58.0,0.257778,0.0,0.000000,35.0,35.0,0.0,1.0,2.0,0.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,81.0,0.0,15.0,0.0,1.0,1.0,0.012346,0.0,0.000000,14.0,14.0,0.0,1.0,3.0,0.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,177.0,0.0,16.0,0.0,0.0,47.0,0.265537,0.0,0.000000,40.0,40.0,0.0,1.0,3.0,0.0
3,mail.printakid.com/www.online.americanexpress....,bad,60.0,0.0,18.0,0.0,1.0,0.0,0.000000,0.0,0.000000,15.0,15.0,0.0,1.0,3.0,0.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,116.0,0.0,19.0,1.0,0.0,21.0,0.181034,0.0,0.000000,23.0,15.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549341,23.227.196.215/,bad,15.0,0.0,14.0,0.0,0.0,11.0,0.733333,11.0,0.785714,3.0,0.0,1.0,0.0,2.0,NaN
549342,apple-checker.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0,1.0,3.0,NaN
549343,apple-iclods.org/,bad,17.0,0.0,16.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.0,0.0,0.0,1.0,3.0,NaN
549344,apple-uptoday.org/,bad,18.0,0.0,17.0,0.0,0.0,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0,1.0,3.0,NaN


### ratio_intHyperlinks

### empty_title

import requests
from bs4 import BeautifulSoup

def check_empty_title(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Sprawdzenie, czy żądanie się powiodło
        
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else ''
        
        if not title.strip():
            return 0  # Tytuł jest pusty
        else:
            return 1  # Tytuł nie jest pusty
    except requests.RequestException as e:
        return 0  # W przypadku błędu traktujemy tytuł jako pusty

check_empty_title(df['URL'][346])

for i in range(len(df)):
    df.loc[i, "empty_title"] = check_empty_title(df.loc[i, "URL"])
df

### domain_in_title

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def check_domain_in_title(url):
    try:
        # Pobierz zawartość strony
        response = requests.get(url)
        response.raise_for_status()
        
        # Parsowanie HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Wyodrębnienie tytułu
        title = soup.title.string if soup.title else ''
        
        # Wyodrębnienie nazwy domeny
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        
        # Usunięcie 'www' z nazwy domeny, jeśli istnieje
        domain = domain.replace('www.', '')
        
        # Sprawdzenie, czy nazwa domeny jest w tytule
        if domain in title:
            return 1
        else:
            return 0
    
    except requests.RequestException as e:
        return 0  # W przypadku błędu zwracamy False


check_domain_in_title(df["URL"][346])

for i in range(len(df)):
    df.loc[i, "domain_in_title"] = check_domain_in_title(df.loc[i, "URL"])
df

5.tld_in_subdomain
6.prefix_suffix (?)
7.shortest_word_host
8.phish_hints
9.nb_hyperlinks
10.ratio_intHyperlinks
11.empty_title
12.domain_in_title